# Importing data and libraries

In [92]:
# %pip install folium
# %pip install geopandas

In [1]:
# importing libraries
import pandas as pd 
import numpy as np
import matplotlib
import seaborn as sns
import os
import folium 
import json
import geopandas as gpd
import pprint


In [2]:
# define path
PATH = 'Data'

In [4]:
# import json file
country_geo_gp=gpd.read_file(os.path.join(PATH, 'Origional Data', 'world-countries.json'))
country_geo_gp.rename(columns={'name':'country'}, inplace=True)

# country_geo_gp[country_geo_gp['name']=='Greece'].plot()

In [5]:
country_geo_gp.shape

(177, 3)

In [6]:
# import the netflix data
df= pd.read_csv(os.path.join(PATH, 'Prepared Data','Netflix_Data_clean.csv'))

# Data wrangling and cleaning

In [7]:
# extract digit grom strings in duration column
df['duration'] = df['duration'].str.extract('(\d+)')

In [8]:
# unstacking the country column, in order to make a choroplath
df['country']=df['country'].str.split(',')

df=df.explode('country')



In [171]:
#removing blank spaces
df['country']=df['country'].str.strip()
df.head()

Unnamed: 0 show_id   type    title      director  \
0           7      s8  Movie  Sankofa  Haile Gerima   
0           7      s8  Movie  Sankofa  Haile Gerima   
0           7      s8  Movie  Sankofa  Haile Gerima   
0           7      s8  Movie  Sankofa  Haile Gerima   
0           7      s8  Movie  Sankofa  Haile Gerima   

                                                cast         country  \
0  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...   United States   
0  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...           Ghana   
0  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...    Burkina Faso   
0  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...  United Kingdom   
0  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...         Germany   

           date_added  release_year rating duration  \
0  September 24, 2021          1993  TV-MA      125   
0  September 24, 2021          1993  TV-MA      125   
0  September 24, 2021          1993  TV-MA      125   
0  September 24, 2021          1993  TV-MA      125   
0  September 24, 2021          1993  TV-MA      125   

                                          listed_in  
0  Dramas, Independent Movies, International Movies  
0  Dramas, Independent Movies, International Movies  
0  Dramas, Independent Movies, International Movies  
0  Dramas, Independent Movies, International Movies  
0  Dramas, Independent Movies, International Movies

In [9]:
#mapping
mapping={
    'United States': 'United States of America',
    'Hong Kong': 'China',
    'West Germany':'Germany',
    'East Germany':'Germany',
    'Soviet Union': 'Russia',  # Replace with the current country name
    'Cayman Islands': 'United Kingdom',  # Replace with the country the territory belongs to
    'Vatican City': 'Italy',
    'Bahamas': 'United States of America',
    'Bermuda': 'United Kingdom',  # Replace with the country the territory belongs to
}


In [10]:
df['country']=df['country'].replace(mapping)# replacing all the country name with mapping

In [169]:
# df_movies to keep only movies for further analysis
# df_shoes to keep only shaws for further analysis


# choropleth map

In [11]:
# Plotting a Choropleth
pd.options.display.max_rows = 4000

data_to_plot = df['country'].value_counts().reset_index()
data_to_plot.columns=['country', 'value']
data_to_plot.head()


country  value
0  United States of America   2133
1                     India    910
2             United States    355
3            United Kingdom    347
4                    Canada    166

In [ ]:
# merging two dataframe(country_geo_gp and data_to_plot)
#inersection=pd.merge(country_geo_gp,data_to_plot,how='outer', on='country', indicator=True)
#inersection[inersection['_merge']=='right_only']

In [14]:
# as soon as you fix all the right_only cases, you need to inner join
inersection_plot=pd.merge(country_geo_gp,data_to_plot,how='inner', on='country', indicator=False)
# inersection_plot

In [15]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [0,-10], zoom_start = 1)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo_gp, #inersection_plot, 
    data = #data_to_plot, 
            inersection_plot[['country','value']], # number of rows must be equal to number of rows in geo_data
    columns = ['country', 'value'], #here dummy is a variable which is a subject to YOUR ANALYSIS
    key_on = 'feature.properties.country', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "title").add_to(map)
folium.LayerControl().add_to(map)
map

In [18]:
# Save the map as an HTML file
map.save(Path, ''"choropleth_map.html")

# Q.n.8

Through this analysis, I have attempted to answer one of the questions: "Which countries produce the most content available on Netflix?" Based on the findings from the map visualization, it is evident that the United States and India are the top producers of content on Netflix.